In [1]:
import pandas, numpy

## functions & options


In [2]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

In [3]:
#define a function to split after the ".". The 1 is for how many word to have. if we have several dot. The [1] is what is after the dot, and [0] what is before
def split(L):
  return L.split(".",1)[0]

# II] Dataframe settings

In [4]:
%%time
path = input_file_directory + "protein_coding_list.txt"
protein_coding_list= pandas.read_csv(path, sep = "\t")
print(protein_coding_list.shape)
protein_coding_list.head()

(22796, 4)
Wall time: 38.1 ms


,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype
1,ENSG00000198888,MT-ND1,4535.0,protein_coding
2,ENSG00000198763,MT-ND2,4536.0,protein_coding
3,ENSG00000198804,MT-CO1,4512.0,protein_coding
4,ENSG00000198712,MT-CO2,4513.0,protein_coding
5,ENSG00000228253,MT-ATP8,4509.0,protein_coding


In [5]:
%%time
#Download the sample labels for normal and primary
path = input_file_directory + "/samples_IDs/samples_normal_tissue.tsv"
normal_tissue_IDs = pandas.read_csv(path, sep = "\t")

path = input_file_directory + "/samples_IDs/samples_primary_tumors.tsv"
primary_tumor_IDs = pandas.read_csv(path, sep = "\t")

#Convert the labels to list
normal_samples_labels = normal_tissue_IDs['sample'].to_list()
primary_samples_labels = primary_tumor_IDs['sample'].to_list()


Wall time: 40 ms


In [6]:
%%time
#Download the big expression data
path = input_file_directory + "/xenabrowser_brut_data/TcgaTargetGtex_rsem_gene_tpm.tsv"
df_gexpr_all = pandas.read_csv(path, sep = "\t", index_col='sample')

print(df_gexpr_all.shape)
df_gexpr_all.head(2)

(60498, 19131)
Wall time: 13min 5s


,GTEX-S4Q7-0003-SM-3NM8M,TCGA-19-1787-01,TCGA-S9-A7J2-01,GTEX-QV31-1626-SM-2S1QC,TCGA-G3-A3CH-11,TCGA-B5-A5OE-01,GTEX-13QIC-0011-R1a-SM-5O9CJ,TCGA-B2-5641-11,GTEX-ZPCL-0126-SM-4WWC8,TARGET-20-PANGDN-09,...,TCGA-FI-A2EY-01,TCGA-55-6985-11,TCGA-EJ-5527-01,TCGA-G3-A25X-01,TCGA-24-2254-01,GTEX-11ZTS-3326-SM-5LU9Y,GTEX-VJYA-0726-SM-4KL1T,GTEX-ZA64-2126-SM-5Q5A8,GTEX-Q2AG-2826-SM-2HMJQ,GTEX-XV7Q-0426-SM-4BRVN
sample,,,,,,,,,,,,,,,,,,,,,
ENSG00000242268.2,-3.4580,-9.9658,0.2998,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-4.0350,-2.0529,-9.9658,-9.9658,-1.9379,1.5165,-9.9658,-2.3884,0.0440,-3.3076
ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [7]:
%%time
# transform data from log2(tpm + 0.001) to log2(tpm + 1)
print(f'min in log2(tpm+0.001) : {df_gexpr_all.min().min()}')
df_gexpr_all = ((2**df_gexpr_all) - 0.001) + 1
print(f'min in tpm+1 : {df_gexpr_all.min().min()}')
df_gexpr_all = numpy.log2(df_gexpr_all) 
print(f'min in log2(tpm+1) : {df_gexpr_all.min().min()}')

# if value is negative, it's zero
df_gexpr_all = (df_gexpr_all).where(df_gexpr_all > 0, 0)
print(f'min in log2(tpm+1) : {df_gexpr_all.min().min()}')
df_gexpr_all.head(2)

min in log2(tpm+0.001) : -9.9658
min in tpm+1 : 0.9999999891070172
min in log2(tpm+1) : -1.5715252422426845e-08
min in log2(tpm+1) : 0.0
Wall time: 1min 52s


,GTEX-S4Q7-0003-SM-3NM8M,TCGA-19-1787-01,TCGA-S9-A7J2-01,GTEX-QV31-1626-SM-2S1QC,TCGA-G3-A3CH-11,TCGA-B5-A5OE-01,GTEX-13QIC-0011-R1a-SM-5O9CJ,TCGA-B2-5641-11,GTEX-ZPCL-0126-SM-4WWC8,TARGET-20-PANGDN-09,...,TCGA-FI-A2EY-01,TCGA-55-6985-11,TCGA-EJ-5527-01,TCGA-G3-A25X-01,TCGA-24-2254-01,GTEX-11ZTS-3326-SM-5LU9Y,GTEX-VJYA-0726-SM-4KL1T,GTEX-ZA64-2126-SM-5Q5A8,GTEX-Q2AG-2826-SM-2HMJQ,GTEX-XV7Q-0426-SM-4BRVN
sample,,,,,,,,,,,,,,,,,,,,,
ENSG00000242268.2,0.124327,0.0,1.157027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.084067,0.310339,0.0,0.0,0.333419,1.948586,0.0,0.250954,1.021457,0.137501
ENSG00000259041.1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [8]:
%%time
#remove dot in ENSEMBL name. Function split created at the beginning
#remove dot in ens name
df_gexpr_all = df_gexpr_all.reset_index()
df_gexpr_all["sample"] = df_gexpr_all["sample"].apply(split)
#see if have duplicate
print('number of duplicated genes:', df_gexpr_all.duplicated('sample').sum())

#Merge with prot coding list to only have protein coding genes
df_gexpr_all.rename(columns = {'sample' : 'ensembl_gene_id'}, inplace = True)
dfmerged = protein_coding_list.merge(df_gexpr_all)
print(dfmerged.shape)
dfmerged.head(2)

number of duplicated genes: 0
(19536, 19135)
Wall time: 49 s


,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype,GTEX-S4Q7-0003-SM-3NM8M,TCGA-19-1787-01,TCGA-S9-A7J2-01,GTEX-QV31-1626-SM-2S1QC,TCGA-G3-A3CH-11,TCGA-B5-A5OE-01,...,TCGA-FI-A2EY-01,TCGA-55-6985-11,TCGA-EJ-5527-01,TCGA-G3-A25X-01,TCGA-24-2254-01,GTEX-11ZTS-3326-SM-5LU9Y,GTEX-VJYA-0726-SM-4KL1T,GTEX-ZA64-2126-SM-5Q5A8,GTEX-Q2AG-2826-SM-2HMJQ,GTEX-XV7Q-0426-SM-4BRVN
0,ENSG00000198888,MT-ND1,4535.0,protein_coding,11.939367,13.128761,12.921486,12.334979,13.336739,10.062848,...,13.939892,12.947882,11.824198,9.856256,13.257247,13.006775,12.282089,13.357037,14.348669,11.665844
1,ENSG00000198763,MT-ND2,4536.0,protein_coding,11.793906,13.079467,12.712815,12.911987,13.418732,9.886523,...,14.390667,13.026673,11.737322,9.358397,13.046770,13.525022,12.655623,13.940492,14.707554,12.865693


In [9]:
#Control of the merge. See value of ENSG00000198712 before merging
print(df_gexpr_all.loc[df_gexpr_all['ensembl_gene_id'] == 'ENSG00000198712']['GTEX-QV31-1626-SM-2S1QC'])

#Control of the merge. See value of ENSG00000198712 after merging. have to be the same
print(dfmerged.loc[dfmerged['ensembl_gene_id'] == 'ENSG00000198712']['GTEX-QV31-1626-SM-2S1QC'])

#release ram
del df_gexpr_all

28551    13.447329
Name: GTEX-QV31-1626-SM-2S1QC, dtype: float64
3    13.447329
Name: GTEX-QV31-1626-SM-2S1QC, dtype: float64


In [10]:
%%time
#Isolate Normal and Primary tissues
##keep the gene_ids
df_part1 = dfmerged.iloc[:,0:4]

##subset
normal_protcoding = dfmerged.iloc[:,4:][normal_samples_labels]
primary_protcoding = dfmerged.iloc[:,4:][primary_samples_labels]

normal_protcoding = pandas.concat([df_part1, normal_protcoding], axis=1)
primary_protcoding = pandas.concat([df_part1, primary_protcoding], axis=1)

print(normal_protcoding.shape)
print(primary_protcoding.shape)
normal_protcoding.head(2)

(19536, 7433)
(19536, 9189)
Wall time: 2.93 s


,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,...,GTEX-ZZPU-0726-SM-5N9C8,GTEX-ZZPU-0926-SM-5GZYT,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000198888,MT-ND1,4535.0,protein_coding,13.135160,14.671255,12.892590,12.626528,13.833999,13.796301,...,12.856694,14.807350,14.324070,14.077683,14.160279,13.710608,12.875992,12.543841,14.608858,14.322870
1,ENSG00000198763,MT-ND2,4536.0,protein_coding,12.621729,14.293172,12.661422,13.083466,14.178278,13.815800,...,13.035472,14.654956,14.349269,14.271173,14.381068,14.243274,13.394134,12.824699,14.249174,14.383067


In [11]:
#test if everything is still good after concat and subset
print(dfmerged['TCGA-ZS-A9CG-01'][12])
print(primary_protcoding['TCGA-ZS-A9CG-01'][12])

13.246248335218185
13.246248335218185


In [12]:
print("Normal tissue")
print("is nan in ensembl:", normal_protcoding['ensembl_gene_id'].isna().sum())
print(' number of duplicate:',normal_protcoding.duplicated('ensembl_gene_id').sum())
print("is nan in symbol:",normal_protcoding['hgnc_symbol'].isna().sum())
print(' number of duplicate:',normal_protcoding.duplicated('hgnc_symbol').sum())
print()
print("Primary Tumor")
print("is nan in ensembl:", primary_protcoding['ensembl_gene_id'].isna().sum())
print(' number of duplicate:',primary_protcoding.duplicated('ensembl_gene_id').sum())
print("is nan in symbol:",primary_protcoding['hgnc_symbol'].isna().sum())
print(' number of duplicate:',primary_protcoding.duplicated('hgnc_symbol').sum())


Normal tissue
is nan in ensembl: 0
 number of duplicate: 154
is nan in symbol: 193
 number of duplicate: 347

Primary Tumor
is nan in ensembl: 0
 number of duplicate: 154
is nan in symbol: 193
 number of duplicate: 347


In [13]:
#see how many duplicates in genes
print('normal shape:', normal_protcoding.shape)
print('number of duplicate:',normal_protcoding.duplicated('ensembl_gene_id').sum())
print()
print('primary shape is:', primary_protcoding.shape)
print('number of duplicate:',primary_protcoding.duplicated('ensembl_gene_id').sum())
print()

#delete duplicates
normal_protcoding.drop_duplicates(subset = "ensembl_gene_id", inplace = True)
primary_protcoding.drop_duplicates(subset = "ensembl_gene_id", inplace = True)

normal shape: (19536, 7433)
number of duplicate: 154

primary shape is: (19536, 9189)
number of duplicate: 154



## A) ENSEMBL

In [14]:
%%time
#Preparation of the global dataframe
protcoding_final = []
data = [normal_protcoding, primary_protcoding]

for df in data:
    #transpose the table 
    df = numpy.transpose(df)

    #Change head column & delete the name of the column index; choose ENSEMBL
    df.columns = df.iloc[0]
    df.columns.name = None
    df = df.reset_index()

    #Delete the first rows & columns rename to merge 
    df = df.drop(df.index[0:4])
    df.rename(columns = {'index' : 'sample'}, inplace = True)

    protcoding_final.append(df)
    
ensembl_normal_protcoding_expr = protcoding_final[0]
ensembl_primary_protcoding_expr = protcoding_final[1]

print('normal shape =', ensembl_normal_protcoding_expr.shape)
print('primary shape =', ensembl_primary_protcoding_expr.shape)
ensembl_normal_protcoding_expr.head()

normal shape = (7429, 19383)
primary shape = (9185, 19383)
Wall time: 1min 46s


,sample,ENSG00000198888,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,...,ENSG00000160678,ENSG00000160679,ENSG00000143553,ENSG00000214193,ENSG00000196182,ENSG00000181817,ENSG00000116885,ENSG00000116898,ENSG00000119535,ENSG00000142694
4,GTEX-1117F-0226-SM-5GZZ7,13.1352,12.6217,12.6248,12.8386,13.9764,13.1288,13.5277,11.9141,12.0782,...,4.35615,5.43295,5.41918,4.68197,4.87574,5.52289,3.28536,5.65136,4.62589,6.44158
5,GTEX-1117F-0426-SM-5EGHI,14.6713,14.2932,14.5996,14.7049,15.7851,15.3795,15.3725,13.0618,13.6228,...,7.80615,4.50081,4.63923,1.97819,4.88608,6.36489,2.70044,6.70865,1.49058,2.9982
6,GTEX-1117F-0526-SM-5EGHJ,12.8926,12.6614,12.501,12.8458,13.9253,13.3518,13.4493,11.6799,12.2199,...,1.30451,5.20871,4.75537,3.03914,5.63578,5.02506,2.43562,5.02729,2.70705,5.58044
7,GTEX-1117F-0626-SM-5N9CS,12.6265,13.0835,11.334,12.693,14.1038,13.4509,13.2863,12.949,12.1296,...,2.93166,5.97382,5.64676,1.89528,5.17953,5.31575,2.79493,5.74817,4.2555,6.38426
8,GTEX-1117F-0726-SM-5GIEN,13.834,14.1783,15.359,14.8495,16.3241,16.1573,15.4759,14.3122,14.4321,...,9.08605,5.14978,4.66736,2.81559,4.08239,4.73987,2.5777,5.04435,4.09594,4.03295


In [15]:
#save to csv
path = 'dataframes_for_input/'
ensembl_normal_protcoding_expr.to_csv(output_file_directory+path+'ensembl_normal_protcoding_expr.tsv',sep = "\t", index=False)
ensembl_primary_protcoding_expr.to_csv(output_file_directory+path+'ensembl_primary_protcoding_expr.tsv',sep = "\t", index=False)

## B) SYMBOLS

In [16]:
# %%time
# #Preparation of the global dataframe
# protcoding_final = []
# data = [normal_protcoding, primary_protcoding]

# for df in data:
#     df = df.dropna(subset=['hgnc_symbol'])
#     #transpose the table 
#     df = numpy.transpose(df)

#     #Change head column & delete the name of the column index; choose ENSEMBL
#     df.columns = df.iloc[1]
#     df.columns.name = None
#     df = df.reset_index()

#     #Delete the first rows & columns rename to merge 
#     df = df.drop(df.index[0:4])
#     df.rename(columns = {'index' : 'sample'}, inplace = True)

#     protcoding_final.append(df)
    
# symbol_normal_protcoding = protcoding_final[0]
# symbol_primary_protcoding = protcoding_final[1]

# print('normal shape =', symbol_normal_protcoding.shape)
# print('primary shape =', symbol_primary_protcoding.shape)
# symbol_normal_protcoding.head()

In [17]:
# #save to csv
# path = 'dataframes_for_input/'
# symbol_normal_protcoding.to_csv(output_file_directory+path+'symbol_normal_protcoding_expr.tsv',sep = "\t", index=False)
# symbol_primary_protcoding.to_csv(output_file_directory+path+'symbol_primary_protcoding_expr.tsv',sep = "\t", index=False)